## Dependencies and Setup

**Remember** to update `voter_data_location`!

In [1]:
!pip install kmodes -Uqq
!pip install 'plotly == 5.1.0' -qq
!pip install jupyter-dash -Uqq

     |████████████████████████████████| 20.6 MB 1.2 MB/s 
     |████████████████████████████████| 1.1 MB 6.8 MB/s 
     |████████████████████████████████| 3.7 MB 53.6 MB/s 
     |████████████████████████████████| 83 kB 2.4 MB/s 
     |████████████████████████████████| 1.8 MB 70.1 MB/s 
     |████████████████████████████████| 357 kB 62.4 MB/s 


In [25]:
# General Imports
import random
from google.colab import drive
from kmodes.kmodes import KModes

# Numerical Programming Imports
import pandas as pd
import numpy as np

# Plotting Imports
import seaborn as sn
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Setup
drive.mount('/content/gdrive')
pd.options.plotting.backend = "plotly"
random.seed(10)

################################################
######  UPDATE THIS FOR YOUR ENVIRONMENT  ######
################################################
voter_data_input_loc = "/content/gdrive/MyDrive/programming/dsaNatCon2021/DSANatCon2021Votes_raw.csv"
clustered_data_output_loc = "/content/gdrive/MyDrive/programming/dsaNatCon2021/DSANatCon2021Votes_withClusters.csv"
################################################
###### ↑  ↑  ↑  ↑  ↑  ↑  ↑  ↑  ↑  ↑  ↑  ↑ ######
################################################

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


## Get Voter Data in Long Format

In [27]:
data = pd.read_csv(voter_data_input_loc)

# Remove wonky test ballots, bunk "Motion #s" have paragraph long ids
data = data[data["Motion #"].str.len() < 15]

In [33]:
# Helper function used in presenting data to plotly heatmap
def toInt(voteChoice):
  if voteChoice == "Yes":
    return 1
  if voteChoice == "No":
    return 0
  return np.nan

# Pull out columns needed for analysis
voters = data[["Delegate", "Motion #", "Vote Choice"]].astype("category")

# Remove ballots with null values
voters = voters[pd.notnull(voters).all(axis=1)].reset_index()

# Add '' to vote choice categories for later use in kmodes analysis
# (library doesn't like NaN)
voters['Vote Choice'] = voters['Vote Choice'].cat.add_categories('')

voters['City'] = voters['Delegate'].apply(
    lambda delegate_comma_city: str(delegate_comma_city).split(',')[1])
voters['Delegate'] = voters['Delegate'].apply(
    lambda delegate_comma_city: str(delegate_comma_city).split(',')[0])

# Rearrange columns to look nice to me
voters = voters.iloc[:, [1,4,2,3]]

voters

,Delegate,City,Motion #,Vote Choice
0,Aaztli C.,Triangle,CB1-A1,Yes
1,Fainan A L.,New York City,CB1-A1,Yes
2,Brendan D.,At-Large,CB1-A1,Yes
3,Nikhil P.,Atlanta,CB1-A1,Yes
4,Zachery H.,Metro Cincinnati & Northern Kentucky,CB1-A1,Yes
...,...,...,...,...
41809,Avir W.,Boston,M80,No
41810,Frankie S.,Metro DC,M80,No
41811,Adam M.,Southeast Wyoming,M80,No
41812,Shepard E.,Colorado Springs,M80,No


## Exploratory Visualization

In [36]:
fig = go.Figure(data=go.Heatmap(
                   z=voters["Vote Choice"].apply(toInt),
                   x=voters["Motion #"],
                   y=voters["Delegate"]))
fig.show()

## Determine "K" for K-modes Clustering

In [14]:
# Pivot to wide form data (each vote is a column, each row is a voter)
voters_wide = voters
voters_wide = voters.pivot(index=["Delegate", "City"], columns="Motion #", values="Vote Choice").fillna('')

# Create matrix for kmodes
voterMatrix = voters_wide.to_numpy()
voterMatrix

array([['', 'Yes', 'No', ..., 'No', 'Yes', 'Yes'],
       ['Yes', 'No', 'No', ..., 'Yes', 'Yes', 'Yes'],
       ['', 'No', 'No', ..., '', 'Yes', ''],
       ...,
       ['Yes', '', 'Yes', ..., '', 'No', 'No'],
       ['Yes', 'No', 'No', ..., 'Yes', '', ''],
       ['Yes', '', 'No', ..., 'Yes', 'No', 'No']], dtype=object)

In [ ]:
k_min = 2
k_max = 21

# Calculating costs for K
cost = []
for cluster in range(k_min, k_max):
    try:
        kmodes = KModes(n_jobs = -1, n_clusters = cluster, random_state = 0)
        kmodes.fit_predict(voterMatrix)
        cost.append(kmodes.cost_)
        print('Cluster initiation: {}'.format(cluster))
    except:
        break

# Converting the results into a dataframe and plotting them
df_cost = pd.DataFrame({'Cluster': range(k_min, k_max), 'Cost': cost}).set_index('Cluster')
df_cost.plot()

# Visually inspect for an elbow to find our K value

Cluster initiation: 2
Cluster initiation: 3
Cluster initiation: 4
Cluster initiation: 5
Cluster initiation: 6
Cluster initiation: 7
Cluster initiation: 8
Cluster initiation: 9
Cluster initiation: 10
Cluster initiation: 11
Cluster initiation: 12
Cluster initiation: 13
Cluster initiation: 14
Cluster initiation: 15
Cluster initiation: 16
Cluster initiation: 17
Cluster initiation: 18
Cluster initiation: 19
Cluster initiation: 20


## Cluster with k-modes

In [15]:
##### UPDATE ME WITH VALUE AT ELBOW OF CHART IN PREVIOUS STEP #####
k_val = 6

# Make our clusters
kmodes = KModes(n_jobs = -1, n_clusters = k_val, random_state = 0)
kmodes.fit_predict(voterMatrix)

# Cost of the clusters created
kmodes.cost_

17797.0

## Attach Cluster Info and Pivot Voter Data to Pseudo Wide Form

In [17]:
# Attach clusters to voter data.
# Why is it so damn hard to just add a column to a multi-indexed dataframe

# Generate mapping of (delegate, city) -> cluster
label_map = {0:'First', 1:'Second', 2:'Third', 3:'Fourth', 4:'Fifth', 5:'Sixth'}
cluster_map = pd.DataFrame(voters_wide.index.to_series())
cluster_map['Cluster'] = pd.Series(np.vectorize(lambda label: label_map[label])(kmodes.labels_)).values
cluster_map = cluster_map.iloc[:, [1]].astype('category')

# Attach
results = voters_wide
results = cluster_map.join(results)

# Sort by cluster
results['Cluster'].cat.reorder_categories(['First', 'Second', 'Third', 'Fourth', 'Fifth', 'Sixth'], inplace=True)
results = results.sort_values('Cluster')

results

,,Cluster,CB1-A1,CB3,CB6,CB7,CB8,M14,M20,M21,M29,M32,M33,M34,M39,M42,M45,M47,M53,M59,M69,M71,M74,M75,M77,M80,NPC4-A1,NPC5-A1,NPC5-A2,NPC5-R5,R1,R14,R18,R20,R20-A9,R23,R26-A12,R29,R30,R32,R38,R38-A14,R5-A2,R8,R8-A5,R8-A6
Delegate,City,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Sean D.,New Orleans,First,Yes,,,,,No,No,No,No,Yes,No,Yes,Yes,Yes,Yes,Yes,No,No,,No,,Yes,Yes,No,Yes,Yes,,,No,Yes,,No,No,No,,No,Yes,No,No,No,No,Yes,No,No
Samuel L.,New York City,First,Yes,No,Yes,,No,Yes,No,No,No,,No,Yes,No,No,No,,No,,Yes,,No,Yes,Yes,Yes,,,No,Yes,Yes,Yes,No,No,No,Yes,No,Yes,Yes,,No,No,No,Yes,No,No
Samuel N.,Lawrence,First,Yes,No,Yes,No,No,Yes,No,No,No,Yes,No,Yes,No,No,,No,No,No,Yes,No,No,Yes,Yes,Yes,Yes,,No,Yes,,Yes,,No,Yes,Yes,No,,Yes,No,No,No,No,Yes,No,No
David R.,Columbus,First,Yes,,,No,No,Yes,No,No,No,Yes,No,Yes,No,No,Yes,,,No,,,,,,,,,No,,Yes,Yes,No,No,No,No,No,No,No,No,No,No,,Yes,No,
Samuel S.,Los Angeles,First,Yes,,Yes,,,,,,No,Yes,No,Yes,Yes,,No,Yes,No,Yes,,,No,No,Yes,Yes,,No,No,Yes,No,Yes,No,,,Yes,No,No,,,No,No,No,,No,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Hadass W.,New York City,Sixth,Yes,,,,Yes,Yes,Yes,Yes,No,Yes,,,No,Yes,Yes,,No,Yes,,No,,,,,Yes,Yes,Yes,,,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,No,No,,,Yes,,
Haifa B.,Orlando,Sixth,Yes,,Yes,Yes,,Yes,Yes,No,Yes,Yes,No,Yes,Yes,Yes,,Yes,No,Yes,Yes,Yes,,Yes,,No,Yes,,,No,,Yes,,Yes,Yes,Yes,,No,No,No,,,No,Yes,No,No
Hannah E.,Suffolk County,Sixth,Yes,No,No,No,Yes,Yes,Yes,Yes,No,Yes,No,No,No,Yes,Yes,Yes,,Yes,Yes,No,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,No,Yes,Yes,Yes,Yes,Yes,Yes,No,No,No,Yes,No,No,Yes,Yes,No


## Plotting

In [18]:
# Big group of heatmaps

voters_clustered = voters.join(results['Cluster'], on=['Delegate', 'City'], sort='Cluster')
first_cluster = voters_clustered[voters_clustered['Cluster'] == 'First']
second_cluster = voters_clustered[voters_clustered['Cluster'] == 'Second']
third_cluster = voters_clustered[voters_clustered['Cluster'] == 'Third']
fourth_cluster = voters_clustered[voters_clustered['Cluster'] == 'Fourth']
fifth_cluster = voters_clustered[voters_clustered['Cluster'] == 'Fifth']
sixth_cluster = voters_clustered[voters_clustered['Cluster'] == 'Sixth']

fig = make_subplots(rows=6, cols=1, subplot_titles=(
    'Cluster 1: %s delegates' % first_cluster['Delegate'].nunique(),
    'Cluster 2: %s delegates' % second_cluster['Delegate'].nunique(),
    'Cluster 3: %s delegates' % third_cluster['Delegate'].nunique(),
    'Cluster 4: %s delegates' % fourth_cluster['Delegate'].nunique(),
    'Cluster 5: %s delegates' % fifth_cluster['Delegate'].nunique(),
    'Cluster 6: %s delegates' % sixth_cluster['Delegate'].nunique(),
))
fig.add_trace(go.Heatmap(
                   z=first_cluster["Vote Choice"].apply(toInt),
                   x=first_cluster["Motion #"],
                   y=first_cluster["Delegate"],
                   ),
              row=1, col=1)
fig.add_trace(go.Heatmap(
                   z=second_cluster["Vote Choice"].apply(toInt),
                   x=second_cluster["Motion #"],
                   y=second_cluster["Delegate"],
                   ),
              row=2, col=1)
fig.add_trace(go.Heatmap(
                   z=third_cluster["Vote Choice"].apply(toInt),
                   x=third_cluster["Motion #"],
                   y=third_cluster["Delegate"],
                   ),
              row=3, col=1)
fig.add_trace(go.Heatmap(
                   z=fourth_cluster["Vote Choice"].apply(toInt),
                   x=fourth_cluster["Motion #"],
                   y=fourth_cluster["Delegate"],
                   ),
              row=4, col=1)
fig.add_trace(go.Heatmap(
                   z=fifth_cluster["Vote Choice"].apply(toInt),
                   x=fifth_cluster["Motion #"],
                   y=fifth_cluster["Delegate"],
                   ),
              row=5, col=1)
fig.add_trace(go.Heatmap(
                   z=sixth_cluster["Vote Choice"].apply(toInt),
                   x=sixth_cluster["Motion #"],
                   y=sixth_cluster["Delegate"],
                   ),
              row=6, col=1)

fig.update_xaxes(categoryorder='category ascending')
fig.update_layout(height=1600, width=1200, title_text="The Six Types of DSA NatCon 2021 Voter")
fig.show()

In [19]:
manual_inspection = results
manual_inspection = manual_inspection.reset_index()

# Aggregate Clusters Per City
clusters_by_city = manual_inspection[['City', 'Cluster']].groupby('City').agg(Count=pd.NamedAgg(column="Cluster", aggfunc="value_counts"))


# HELPFUL INSPECTIONS

def lookup_delegate(name):
  return manual_inspection[manual_inspection['Delegate'].str.contains(name)]

def lookup_city(city):
  return manual_inspection[manual_inspection['City'].str.contains(city)]

def cluster_counts_for_city(city):
  return clusters_by_city[clusters_by_city.index.get_level_values(0).str.contains(city)]

def city_counts_for_cluster(cluster):
  return clusters_by_city[clusters_by_city.index.get_level_values(1).str.contains(cluster)]

#city_counts_for_cluster('Fifth').sort_values('Count', ascending=False).head(10)
clusters_by_city.reset_index()

,City,Cluster,Count
0,Akron,First,2
1,Akron,Second,1
2,Akron,Fourth,1
3,Albuquerque,First,1
4,Albuquerque,Fifth,1
...,...,...,...
410,Winston-Salem,Fifth,2
411,Worcester,First,2
412,Worcester,Sixth,1
413,Yolo County,Second,1


In [20]:
clusters_by_city_plot = clusters_by_city.reset_index()

# Sunburst Plot
fig = px.sunburst(clusters_by_city_plot,
                  path=['City', 'Cluster'],
                  values='Count',
                  color='Cluster',
                  title='Clusters by City')

fig.show()


In [21]:
fig = px.sunburst(clusters_by_city.reset_index(),
                  path=['Cluster', 'City'],
                  values='Count',
                  color='Cluster',
                  title='Cities by Cluster')
fig.show()

In [ ]:
# Clusters as percent of chapter delegates
clusters_as_percent_of_city = clusters_by_city.groupby('City').apply(lambda x: 100 * x / float(x.sum()))
clusters_as_percent_of_city['Percent'] = clusters_as_percent_of_city['Count']
clusters_as_percent_of_city['Delegates'] = clusters_by_city['Count']
clusters_as_percent_of_city = clusters_as_percent_of_city.reset_index()


fig = go.Figure(data=[go.Table(
    header=dict(values=list(['Chapter','Cluster','Number of Delegates','Percent of Chapter']),
                fill_color='paleturquoise',
                align='left'),
    cells=dict(values=[clusters_as_percent_of_city['City'], clusters_as_percent_of_city['Cluster'], clusters_as_percent_of_city['Delegates'], clusters_as_percent_of_city['Percent']],
               fill_color='lavender',
               align='left'))
])

fig.show()

## Write Out Clustering Results

In [ ]:
voters_clustered.to_csv(clustered_data_output_loc)